In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

In [2]:
# 데이터 셋
# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, data):
        self.features, self.labels = data

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]
        return feature, label

training_data = (torch.randint(0, 10, (100000, 4)) / 10, torch.randint(0, 10, (100000, 1)) / 10)
training_dataset = CustomDataset(training_data)

test_data = (torch.randint(0, 10, (10000, 4)) / 10, torch.randint(0, 10, (10000, 1)) / 10)
test_dataset = CustomDataset(test_data)

In [3]:
# 데이터 로더
batch_size = 64

train_dataloader = DataLoader(training_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [4]:
device = 'cuda' if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(4, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
    
model = CustomModel().to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
print(model)

CustomModel(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    items = []
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {correct:>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct

In [6]:
epochs = 50
acc = 0
accs = []

start = time.time()
for t in range(epochs):
    clear_output()
    print(f"Epoch {t+1} {acc}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    acc = test(test_dataloader, model, loss_fn)
    accs.append(acc)
    if acc > 90:
        print(f'stop epoch {t+1}')
        epochs = t + 1
        break
end = time.time()

print(f'time {end - start} sec')

plt.figure(figsize=(10, 6))
plt.title('Training Loss over Epochs')
plt.xlabel('epochs')
plt.ylabel('acc')
plt.grid(True)
plt.plot(list(range(0, epochs)), accs, marker='o', linestyle='-', color='b', label='Training Loss')
plt.show()

print("Done!")

Epoch 9 0.6981
-------------------------------
loss: 0.072979  [   64/100000]
loss: 0.090930  [ 6464/100000]
loss: 0.080654  [12864/100000]
loss: 0.084754  [19264/100000]
loss: 0.068134  [25664/100000]
loss: 0.110934  [32064/100000]
loss: 0.090445  [38464/100000]
loss: 0.081001  [44864/100000]
loss: 0.085558  [51264/100000]
loss: 0.077768  [57664/100000]
loss: 0.074108  [64064/100000]
loss: 0.082185  [70464/100000]
loss: 0.098959  [76864/100000]
loss: 0.093694  [83264/100000]
loss: 0.088478  [89664/100000]


KeyboardInterrupt: 